In [2]:
import casadi as ca
import numpy as np
import math
import os

### Import track data

In [4]:
file_paths = {"veh_params_file": "racecar.ini"}
#file_paths["module"] = os.path.dirname(os.path.abspath(__file__))
file_paths["module"] = globals()['_dh'][0]
file_paths["track_name"] = "london_track"
file_paths["track_file"] = os.path.join(file_paths["module"], "track", file_paths["track_name"] + ".csv")

track_data_csv = np.loadtxt(file_paths["track_file"], comments='#', delimiter=',')

centerline = track_data_csv[:, 0:2]
w_r2c = track_data_csv[:, 2]        # right to center distance
w_l2c = track_data_csv[:, 3]        # left to center distance

laps= 1
centerline= np.tile(centerline, (laps,1))
w_r2c= np.tile(w_r2c,laps)
w_l2c= np.tile(w_l2c,laps)

track_ref= np.column_stack((centerline,w_r2c,w_l2c))

kappa = ca.interpolant('kappa', 'linear', [track_ref.shape[0]], track_ref[:, 0])

# Consider reverse running
reverse: bool = False
if reverse:
    track_ref= np.flipud(track_ref)

### Direct Gauss_Legendre Collocation & Lagrange polynomials

In [ ]:
d=3
collocate_points= np.append(0, ca.collocation_points(d, 'legendre'))
C= np.zeros((d+1, d+1))
D= np.zeros(d+1)
B= np.zeros(d+1)

for i in range (d+1):
    p= np.poly1d([1])
    for j in range (d+1):
        if j != i:
            p *= np.poly1d([1, -collocate_points[j]])/ (collocate_points[i]- collocate_points[j])

    D[i]= p(1.0)

    dp= np.polyder(p)
    for j in range (d+1):
        C[i,j]= dp(collocate_points[j])

    intp= np.polyint(p)
    B[i]= intp(1.0)


### Declare states and control variables

In [ ]:
# States variables
v= ca.SX.sym('v')               #velocity
beta= ca.SX.sym('beta')         #slip angle
omega= ca.SX.sym('omega')       #yaw rate
n = ca.SX.sym('n')              #lateral distance to ref line (left:+)
xi = ca.SX.sym('xi')            #relative angle to tangent on ref line
x = ca.vertcat(v, beta, omega, n, xi)

# Control variables
delta= ca.SX.sym('delta')
F_drive= ca.SX.sym('F_drive')
F_brake= ca.SX.sym('F_brake')
gamma_y= ca.SX.sym('gamma_y')
u= ca.vertcat(delta, F_drive, F_brake, gamma_y)

### Define parameters

In [ ]:
# parameters
veh= {"wheelbase_front": 1.6,
      "wheelbase_rear": 1.4,
      "wheelbase": 3,
      "twf": 1.6,
      "twr": 1.6,
      "cog_z": 0.38,
      "J_z": 1200.0,
      "liftcoeff_front": 0.45,
      "liftcoeff_rear": 0.75,
      "k_brake": 0.6,
      "k_drive": 0.0,
      "k_roll": 0.5,
      "t_delta": 0.2,
      "t_drive": 0.05,
      "t_brake": 0.05,
      "power_max": 230000.0,
      "f_drive_max": 7000.0,
      "f_brake_max": 20000.0,
      "delta_max": 0.26,    #15deg
      "g": 9.8,
      "mass": 1200,
      "v_max": 25,
      "dragcoeff": 0.75}

tire= {"c_roll": 0.013,
       "F_z0": 3000.0,
       "B_f": 10.0,
       "C_f": 2.5,
       "eps_f": -0.1,
       "E_f": 1.0,
       "B_r": 10.0,
       "C_r": 2.5,
       "eps_r": -0.1,
       "E_r": 1.0,
       "mu": 1.0}

# scaling the states & control variables
v_s= v* 1.0
beta_s= beta* 1.0
omega_s= omega* 1.0
n_s= n* 1.0
xi_s= xi* 1.0
delta_s= delta* 1.0
F_drive_s= F_drive* 1.0
F_brake_s= F_brake* 1.0
gamma_y_s= gamma_y* 1.0

### Construct state equations

In [ ]:
# rolling resistance
F_xroll_f = 0.5 * tire["c_roll"] * veh["mass"]* veh["g"] * veh["wheelbase_front"] / veh["wheelbase"]
F_xroll_r = 0.5 * tire["c_roll"] * veh["mass"]* veh["g"] * veh["wheelbase_rear"] / veh["wheelbase"]
F_xroll= tire["c_roll"]* veh["mass"]* veh["g"]

# drag force
F_xdrag= veh["dragcoeff"]* math.pow(v_s, 2)

# Longitudinal tire force F_x
F_x_fl= 0.5* veh["k_drive"]* F_drive_s+ 0.5* veh["k_brake"]* F_brake_s- F_xroll_f
F_x_fr= F_x_fl
F_x_rl= 0.5* (1- veh["k_drive"])* F_drive_s+ 0.5* (1- veh["k_brake"])* F_brake_s- F_xroll_r
F_x_rr= F_x_rl

# static normal tire forces
F_zstat_f= 0.5 * veh["mass"] * veh["g"] * veh["wheelbase_front"] / veh["wheelbase"]
F_zstat_r= 0.5 * veh["mass"] * veh["g"] * veh["wheelbase_rear"] / veh["wheelbase"]

# dynamic normal tire force
F_zlift_f = 0.5* veh["liftcoeff_front"]* math.pow(v_s, 2)
F_zlift_r = 0.5* veh["liftcoeff_rear"]* math.pow(v_s, 2)

# load transfers
F_zloadtrans_fl= -0.5* veh["cog_z"]/ veh["wheelbase"]* (F_drive_s+ F_brake_s- F_xdrag- F_xroll)- veh["k_roll"]* gamma_y_s
F_zloadtrans_fr= -0.5* veh["cog_z"]/ veh["wheelbase"]* (F_drive_s+ F_brake_s- F_xdrag- F_xroll)+ veh["k_roll"]* gamma_y_s
F_zloadtrans_rl= -0.5* veh["cog_z"]/ veh["wheelbase"]* (F_drive_s+ F_brake_s- F_xdrag- F_xroll)- (1- veh["k_roll"])* gamma_y_s
F_zloadtrans_rr= -0.5* veh["cog_z"]/ veh["wheelbase"]* (F_drive_s+ F_brake_s- F_xdrag- F_xroll)+ (1- veh["k_roll"])* gamma_y_s

F_z_fl= F_zstat_f+ F_zlift_f+ F_zloadtrans_fl
F_z_fr= F_zstat_f+ F_zlift_f+ F_zloadtrans_fr
F_z_rl= F_zstat_r+ F_zlift_r+ F_zloadtrans_rl
F_z_rr= F_zstat_r+ F_zlift_r+ F_zloadtrans_rr

# sideslip angle
alpah_fl= delta_s- ca.atan((veh["wheelbase_front"]*omega_s+v_s* ca.sin(beta_s))/
                           (v_s*ca.cos(beta_s)- 0.5*veh["twf"]*omega_s))
alpah_fr= delta_s- ca.atan((veh["wheelbase_front"]*omega_s+v_s* ca.sin(beta_s))/
                           (v_s*ca.cos(beta_s)+ 0.5*veh["twf"]*omega_s))
alpah_rl= ca.atan((veh["wheelbase_rear"]*omega_s- v_s* ca.sin(beta_s))/
                  (v_s*ca.cos(beta_s)- 0.5*veh["twr"]*omega_s))
alpah_rr= ca.atan((veh["wheelbase_rear"]*omega_s- v_s* ca.sin(beta_s))/
                  (v_s*ca.cos(beta_s)+ 0.5*veh["twr"]*omega_s))

# Lateral tire force F_y
F_y_fl= tire["mu"]* F_z_fl* (1+ tire["eps_f"]* F_z_fl/ tire["F_z0"])\
        * ca.sin(tire["C_f"]* ca.atan(tire["B_f"]* alpah_fl-
                                      tire["E_f"]*(tire["B_f"]* alpah_fl- ca.atan(tire["B_f"]* alpah_fl))))
F_y_fr= tire["mu"]* F_z_fr* (1+ tire["eps_f"]* F_z_fr/ tire["F_z0"])\
        * ca.sin(tire["C_f"]* ca.atan(tire["B_f"]* alpah_fr-
                                      tire["E_f"]*(tire["B_f"]* alpah_fr- ca.atan(tire["B_f"]* alpah_fr))))
F_y_rl= tire["mu"]* F_z_rl* (1+ tire["eps_f"]* F_z_rl/ tire["F_z0"])\
        * ca.sin(tire["C_f"]* ca.atan(tire["B_f"]* alpah_rl-
                                      tire["E_f"]*(tire["B_f"]* alpah_rl- ca.atan(tire["B_f"]* alpah_rl))))
F_y_rr= tire["mu"]* F_z_rr* (1+ tire["eps_f"]* F_z_rr/ tire["F_z0"])\
        * ca.sin(tire["C_f"]* ca.atan(tire["B_f"]* alpah_rr-
                                      tire["E_f"]*(tire["B_f"]* alpah_rr- ca.atan(tire["B_f"]* alpah_rr))))

# Acceleration
ax = (F_x_r + F_x_r + (F_x_f + F_x_f) * ca.cos(delta) - (F_y_fl + F_y_fr) * ca.sin(delta)
          - veh["dragcoeff"] * v ** 2) / veh["mass"]
ay = ((F_x_f + F_x_f) * ca.sin(delta) + F_y_rl + F_y_rr + (F_y_fl + F_y_fr) * ca.cos(delta)) / veh["mass"]

# Derivatives
k_curv= ca.SX.sym("k_curv")

SF= (1.0- n*k_curv)/(v* ca.cos(xi_s+ beta_s))
dv= (SF/veh["mass"])* ((F_x_r+ F_x_r)* ca.cos(beta_s)+ (F_x_r+ F_x_r)* ca.cos(delta_s- beta_s)+
                       (F_y_rl+ F_y_rr)* ca.sin(beta_s)- (F_y_fl+ F_y_rr)* ca.sin(delta_s- beta_s)-
                       F_xdrag* ca.cos(beta_s))
dbeta= SF* (-omega_s+ 1/(veh["mass"]* v_s)* (-(F_x_r+ F_x_r)* ca.sin(beta_s)+ (F_x_f+ F_x_f)* ca.sin(delta_s-beta_s)+
                                             (F_y_rl+ F_y_rr)* ca.cos(beta_s)+ (F_y_fl+ F_y_fr)* ca.cos(delta_s-beta_s)+
                                             F_xdrag* ca.sin(beta_s)))
domega= SF/veh["J_z"]* ((F_x_r- F_x_r)* veh["twr"]/2- (F_y_rl+ F_y_rr)* veh["wheelbase_rear"]+
                        ((F_x_f- F_x_f)* ca.cos(delta_s)+ (F_y_fl- F_y_fr)* ca.sin(delta_s))*veh["twf"]/2+
                        ((F_y_fl+ F_y_fr)* ca.cos(delta_s)+ (F_x_f+ F_x_f)* ca.sin(delta_s))*veh["wheelbase_front"])
dn= SF* v_s* ca.sin(xi_s+ beta_s)
dxi= SF* omega_s- k_curv

dx= ca.vertcat(dv, dbeta, domega, dn, dxi)

###

In [ ]:
# continuous time dynamics
f_dyn = ca.Function('f_dyn', [x, u, k_curv], [dx, SF], ['x', 'u', 'k_curv'], ['dx', 'SF'])

# longitudinal tire forces [N]
f_Fx = ca.Function('f_Fx', [x, u], [F_x_fl, F_x_fr, F_x_rl, F_x_rr],
                   ['x', 'u'], ['F_x_f', 'F_x_r'])
# lateral tire forces [N]
f_Fy = ca.Function('f_Fy', [x, u], [F_y_fl, F_y_fr, F_y_rl, F_y_rr],
                   ['x', 'u'], ['F_y_fl', 'F_y_fr', 'F_y_rl', 'F_y_rr'])
# vertical tire forces [N]
f_Fz = ca.Function('f_Fz', [x, u], [F_z_fl, F_z_fr, F_z_rl, F_z_rr],
                   ['x', 'u'], ['F_z_fl', 'F_z_fr', 'F_z_rl', 'F_z_rr'])

# longitudinal and lateral acceleration [m/s²]
f_a = ca.Function('f_a', [x, u], [ax, ay], ['x', 'u'], ['ax', 'ay'])

### Specify boundaries

In [ ]:
# Control Boundaries
delta_min= -veh["delta_max"]
delta_max= veh["delta_max"]
F_drive_min= 0.0
F_drive_max= veh["f_drive_max"]
F_brake_min= 0.0
F_brake_max= - veh["f_brake_max"]
gamma_y_min= -np.inf
gamma_y_max= np.inf

#State Boundaries
v_min= 0.1
v_max= veh["v_max"]
beta_min= -np.pi/2
beta_max= np.pi/2
omega_min= -np.pi/2
omega_max= np.pi/2
xi_min= -np.pi/2
xi_max= np.pi/2
n_min= -w_r2c(0)+ 3.4/2
n_max= w_l2c(0)- 3.4/2

### Initialize empty NLP

In [ ]:
num_x= x.size()[0]
num_u= u.size()[0]

J= 0
w=[]
w0= []
min_w= []
max_w= []
g=[]
min_g= []
max_g= []

x_opt= []
u_opt= []

SF_opt= []

Xk = ca.MX.sym('X0', num_x)
w.append(Xk)
min_w.append([v_min, beta_min, omega_min, n_min, xi_min])
max_w.append([v_max, beta_max, omega_max, n_max, xi_max])

w0.append([0.0]*num_x)
x_opt.append(Xk)

### Start to loop each segment

In [9]:
num_seg= track_ref.shape[0]
step= 3
for k in range (num_seg):
    Uk= ca.MX.sym('U_'+ str(k), num_u)
    w.append(Uk)
    min_w.append([delta_min, F_drive_min, F_brake_min, gamma_y_min])
    max_w.append([delta_max, F_drive_max, F_brake_max, gamma_y_max])
    w0.append([0.0]* num_u)

    Xc= []
    for j in range (d):
        Xkj= ca.Mx.sym('X_'+ str(k)+ '_'+ str(j), num_x)
        Xc.append(Xkj)
        w.append(Xkj)
        min_w.append([-np.inf]* num_x)
        max_w.append([np.inf]* num_x)

        w0.append([0.0]* num_x)

    XK_end= D[0]* Xk

    for j in range(1, d+1):
        xp= C[0, j]*Xk
        for r in range(d):
            xp += C[r+1, j]* Xc[r]

        kappa_colloc= kappa(k+collocate_points[j])

        fj, qj= f_dyn(Xc[j-1], Uk, kappa_colloc)
        g.append(step*fj- xp)
        min_g.append([0.0]* num_x)
        max_g.append([0.0]* num_x)
        
        Xk_end= Xk_end+ D[j]* Xc[j-1]
        J= J+ B[j]* qj* step
        
        SF_opt.append(B[j]* qj* step)
        
        #update states in next step
        Xk = ca.MX.sym('X_'+ str(k + 1), num_x)
        w.append(Xk)
        
        n_min= -w_r2c(k+1)+ 3.4/2
        n_max= w_l2c(k+1)- 3.4/2
        min_w.append([v_min, beta_min, omega_min, n_min, xi_min])
        max_w.append([v_max, beta_max, omega_max, n_max, xi_max])
        w0.append([0.0]*num_x)
        
        g.append(Xk_end- Xk)
        min_g([0.0]* num_x)
        max_g([0.0]* num_x)
        
        







NameError: name 'num_u' is not defined